# Minimum-Cost Flow Redistricting Assignment Algorithm
## Examining An Impossibility Theoorem regarding Gerrymandering

The practice of gerrymandering, or the manipulation of electoral district boundaries for political gain, has long been a contentious issue in democratic governance. This Notebook builds upon the foundational concepts laid out in "An Impossibility Theorem for Gerrymandering" by Alexeev and Mixon, refining the established formula to enhance the analysis of electoral districting. Our research develops a sophisticated simulation model to evaluate the implications of various districting strategies on electoral fairness.

## Key parameters influencing district shapes and sizes include:
- **Delta (δ)**: Allowable deviation in population size among districts.
- **Gamma (γ)**: Standard for geographic compactness.
- **Proportion of Voters (p0)**: Ratio of voters from two major parties.
- **Grid Size (n)**: Size of the grid.
- **District Count (k)**: Number of districts.


In [68]:
import pandas as pa
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from ortools.graph import pywrapgraph
import random
import os
from multiprocessing import Pool

### Monitor Simulation. 
#### Set to 1 if you want detailed logs of every function's behaviour

In [69]:
log_flag = 0
print_graphics = 0

In [70]:
def print_info(data, name):
    if isinstance(data, list):
        print(' % ',name,' ',data,' -----type $len: ', len(data),' ', type(data),' * ')
    else:
        print(' % ',name,' ',data,' -----type ', type(data),' * ')

## Create a grid
For cross reference to Google's documentation, the census blocks are the "workers" and the centers of the districts are the "tasks."

Cost being minimized is the product of the (distance from the census block's center to the location of the center to which it is assigned by the algorithm) and (the population of the block).

Source supply is the overall population.

Capacity of each arc from the source to a census block is the population of that census block.

Capacity of each arc from a census block to a center is the population of that census block.

Capacity of each arc from a center to the sink is the ideal district size plus an allowed tolerance. How to set the tolerance is a matter of opinion. A low setting is probably preferable -- perhaps even just going with no tolerance at all.

Sink supply is the negative of the overall population.

After the algorithm is run, 0-population census blocks remain unassigned to a district because the 0 population means the "flow" across their arc is 0.

The algorithm allows a block's population to be split between districts. If this happens the block is actually assigned to the district which received the greatest share of the population by the algorithm.

V3 Update Notes:
    implements functions to calculate pp compactness and effiency gap of the districts (added voter prefernce)

In [71]:
def setup(districts,grid_size,district_centers,pops): 
    """
    Inputs: # of districts, size of grid, predeterminded district centers, total population
    Purpose: This block establishes the parameters for the situation and creates the node and edge inputs
             for the optimization procedure.
    Outputs: A tuple containing start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop
    """
    


    if log_flag == 1:
        print('in setup')
        print(' Inputs-setup \n')
        print_info(districts, 'districts')
        print_info(grid_size, 'grid_size')
        print_info(district_centers, 'district_centers')
        print_info(pops, 'pops')
        print(' end inputs \n')
    
    tol = 1 # tolerance for population difference between districts

    #Build the list holding the node labels corresponding to the sources for the graph's edges
    start_nodes = [0]*(grid_size*grid_size)   #The initial source is labeled 0 and is at the tail of edges which 
                                              #terminate at
                                              #each census block's node
    for i in range(grid_size*grid_size):
        start_nodes = start_nodes+[i+1]*districts

    for j in range(districts):
        start_nodes = start_nodes+[(grid_size*grid_size)+1+j]
        
    #Now build the list holding the node labels corresponding to the sinks for each of the graph's edges

    end_nodes=[]
    for i in range(grid_size*grid_size):
        end_nodes=end_nodes+[i+1]
    
    for j in range(grid_size*grid_size):
        for k in range(districts):
            end_nodes=end_nodes+[(grid_size*grid_size)+1+k]
        
    end_nodes=end_nodes+[(grid_size*grid_size)+districts+1]*districts

    #Now create a list of each edge's capacity.  For edges connected to census block nodes, this will be the
    #population of the census block.
    #For the edges connecting each district to the final sink node, this will be the sum of the districts'
    #populations divided by the number of districts -- i.e., the ideal district size.

    #Now use the populations passed to the function to create edge capacities.

    total_pop = np.sum(np.array(pops))
        
    capacities = np.array(pops).flatten().tolist()

    for i in np.array(pops).flatten().tolist():
        for j in range(districts):
            capacities = capacities+[i]

    for i in range(districts):
        capacities = capacities+[int(total_pop/districts)+tol]#last value here is a tolerance for how much a district
                                                              #can exceed its ideal size

    #A final set-up step involves attaching costs to each edge in the graph.
    #The costs will be the distances between each census block's centroid and the centroids of the possible
    #districts to which it could be assigned.
    #For the purposes of this example, I'm assuming census blocks have centroids located at lattice points
    #spaced 1 unit apart.
    

    #To note: edges from the initial source AND to the final sink have no cost -- and so will be given a cost of 0.

    costs = [0]*(grid_size*grid_size)

    #To calculate distances, we need to establish the locations of the district centers.

    for i in range(grid_size):  #These are the rows of the census block grid
        for j in range(grid_size):   #These are the columns of the census block grid
            for k in range(districts):
                costs = costs+[int((((i-district_centers[k][0])**2)+((j-district_centers[k][1])**2)))]

    costs = costs+[0]*districts

    #Supplies are 0 for each node except the initial source and final sink, each of which has a value 
    #corresponding to the total population to distribute.

    supplies = [int(total_pop)]+((grid_size*grid_size)+districts)*[0]+[-int(total_pop)]

    #This is the source node's label

    source = 0

    #This is the sink node's label

    sink = districts+(grid_size*grid_size)+1
    
    if log_flag == 1:
        print('Out of setup')      
        print(' output-setup \n')
        print_info(start_nodes, 'start_nodes')
        print_info(end_nodes, 'end_nodes')
        print_info(capacities, 'capacities')
        print_info(costs, 'costs')
        print_info(supplies, 'supplies')
        print_info(source, 'source')
        print_info(sink, 'sink')
        print_info(pops, 'pops')
        print_info(total_pop, 'total_pop')
        print(' end outputs \n')

    return(start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop)

### District Generation Algorithm

The districting system partitions the grid into districts based on voter locations, ensuring compliance with the one-person-one-vote principle and compactness measures. The algorithm iteratively adjusts district centers and optimizes according to the efficiency gap and compactness metrics.


In [72]:
def optimize(start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size):
    
    """
    Inputs: start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size 
    Purpose: Uses a min cost flow algorithm to optimize districting  
    Outputs: returns districting assignments
    """
    if log_flag == 1:
        print('in optimize')
    

    #Create empty DataFrame to hold outputs later
    Block_Assignments=pa.DataFrame(columns=['DIST_NO','ASSIGN_POP','ACTUAL_POP'])

    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for i in range(len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                capacities[i], costs[i])

    # Add node supplies.
    for i in range(len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])

    flag = 0    
    
    # Find the minimum cost flow between source and sink.
    if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
        #print('Total cost = ', min_cost_flow.OptimalCost())
        #print('\n')
        for arc in range(min_cost_flow.NumArcs()):

          # Can ignore arcs leading out of source or into sink.
          if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:
        
            if min_cost_flow.Flow(arc) > 0:
          
              #Add a way to check for those instances where a census block's population is split
              #by the algorithm and assigned to more than one block.  
              #Assign the census block to the district which optimally receives the greatest
              #fraction of its population.
            
              if min_cost_flow.Capacity(arc)==min_cost_flow.Flow(arc):
                grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                        (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
            
              else:
            
                if flag==0:
                    grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                        (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                    Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
                
                    flag=min_cost_flow.Flow(arc)
                
                else: 
                    if Block_Assignments.loc[min_cost_flow.Tail(arc),'ASSIGN_POP']<min_cost_flow.Flow(arc):
                        grid.nodes[(int(((min_cost_flow.Tail(arc)-1)/grid_size)),
                            (min_cost_flow.Tail(arc)-1)%grid_size)]["district"]=min_cost_flow.Head(arc)
            
                        Block_Assignments.loc[min_cost_flow.Tail(arc)]=[min_cost_flow.Head(arc),min_cost_flow.Flow(arc),
                                                            min_cost_flow.Capacity(arc)]
                
                        flag = flag+min_cost_flow.Flow(arc)
                
                    else:
                        flag = flag+min_cost_flow.Flow(arc)
                    
                    if flag==min_cost_flow.Capacity(arc):
                        flag = 0
                
    else:
        print('There was an issue with the min cost flow input.')
        
    if log_flag == 1:
        print('out of optimize')

    return(Block_Assignments)

### Utitls

In [73]:
def get_pp_compactness(perimeter, area):
    """
    Inputs: perimeter and area of a districts
    Purpose: calculates ppc of districts 
    Outputs: ppc for the given districts
    """
    if log_flag == 1:
        print('in pp_compact')
        
    dist_pp_compactness = {}
    # go through a list of permeters and ares for each district
    for p_key in perimeter:
        for a_key in area:
            if p_key == a_key:
                compactness = (4 * 3.14 * int(area[a_key][0]))/(perimeter[p_key])**2
                dist_pp_compactness.setdefault(a_key,[]).append(compactness)

    if log_flag == 1:
        print('out of pp_compact')
        
    return dist_pp_compactness

In [74]:
def grid_setup(grid_s, dist, n,r,c,p_0):
    """
    Inputs: grid_size, number of districts, n, r, c, proportion of voters 0
    Purpose: sets up a grid with determined population size for each node 
    Output: districts, grid_size, district_centers, pops, grid
            a grid with population voter pref baked in 
    """
    if log_flag == 1:
        print('in grid setup')
        
    #Here we set up the grid on which the districts will be drawn
    grid_size = grid_s
    # TODO: does not need to return districts in this function
    districts = dist # number of districts
    #District centers can be specified in a variety of ways.
    #The following gives a regular horizontal spacing perturbed by a random factor, and then randomly assigns the
    #location vertically.

    district_centers = []
    spacing = int(grid_size/districts)
    for j in range(districts):
        district_centers = district_centers + [[(j+1)*spacing+random.randint(-1,1),random.randint(0,grid_size-1)]]

    #print(district_centers)

    #Now we establish the population distribution on the grid

    pops = []
    row = []
    for i in range(grid_size):
        for j in range(grid_size):
            #row = row+[i+1]                      #Use this for systematic population assignments
            #row = row+[random.randint(1,10)]     #Use this for random population assignments
            row = row+[1]                         #Use this for a population of 1 at each node
        pops = pops+[row]
        row = []

    total_pop = np.sum(np.array(pops))

    #print('Total population across all blocks is: ',total_pop)

    #And now we create the grid graph and assign the populations to the nodes

    grid = nx.grid_graph([grid_size,grid_size])
    count = 0

    for node in grid.nodes():
        grid.nodes[node]["population"]=pops[node[0]][node[1]]


    add_party_preference(grid, n, grid_size, p_0)
    
    
        
    if log_flag == 1:
        print('out of grid setup')

    return districts, grid_size, district_centers, pops, grid


In [75]:
def add_party_preference(grid, n, grid_size, p_0):
    """
    Inputs: grid of voters, n, grid size, proportion of voters a to b
    Purpose: Function adds voter party preference to each node in the graph 
    Output: No output, voter pref gets mapped to grid passed by grid_setup
    """
    if log_flag == 1:
        print('in party pref')
    squares_with_id = [] #index will serve as id
    
    all_nodes = list(grid.nodes())
    all_nodes_processed = [] #used to test zoom mechanism 
    voter_arr_ind = 0
    row_tracker = 0 # to keep track of the rows within square counting
    square = []
    voter_arr_ind = 0
    vertical_mover = 0
    for i in range(n):
        if i != 0:
            vertical_mover += r*grid_size
        for m in range(n):
            row_tracker = (c*m) + vertical_mover
            #get sqaures at each level
            square = []            
            for j in range(r):
                row = all_nodes[row_tracker:row_tracker+c]
                square = square + row
                row_tracker += grid_size
            all_nodes_processed += square
            #add logic for clumping
            #if clump
            # add two squares together set squares to that
            voter_ratios = np.ones(c*r)
            
            #input porportions here
            proportion_of_a = math.floor((c*r)*p_0)
            voter_ratios[:proportion_of_a] = 0
            #print(voter_ratios)
            np.random.shuffle(voter_ratios)
            #print(voter_ratios)
            for node in square:    
                grid.nodes[node]['voter_pref'] = voter_ratios[voter_arr_ind] # 0 is blue 1 is brown
                voter_arr_ind = voter_arr_ind + 1

            squares_with_id.append(square)       
            voter_arr_ind = 0
            square = []
            


    
    missed_nodes = list(set(all_nodes).difference(all_nodes_processed))
    if missed_nodes:   
        print("missed_nodes = " + str(missed_nodes))
        os._exit(0)
    if log_flag == 1:   
        print('out of party pref')


In [76]:
def dist_assgin(iterations, grid_size, dist_num,n,r,c,p_0):
    """
    Inputs: number iterations for assignment exploration, grid size, number of districts, n, r, c, proportion of voters a to b 
    Purpose: Here we find optimal district assignments by iterating through the optimization process, updating
             district centers by finding centroids of the current step's assignments.
    Output: districts assignemnts per itertation in a dictionary, polsby popper compactness for each iteration 
    """
    break_point = 0
    if log_flag == 1:    
        print('in dist assign')
    districts, grid_size, district_centers, pops, grid = grid_setup(grid_size,dist_num,n,r,c, p_0)
    # TODO: does not need to return districts in this function

    #Establish the number of iterations to carry out
    pp_each_iter = {}
    district_data_each_iteration = {}

    

    for l in range(iterations):

        win_count = {}
        state_voterPrefs_as_row = []
        # district_data_each_iteration.setdefault(l, []).append(win_count)
        # district_data_each_iteration[l].append(l)  # Add the iteration number

        
        # if figures are produced stop iterating
        if break_point == 1:
            break
            
        # reshuffle district centers after first run
        if l!=0:
            district_centers_new=[]
            for i in range(districts):
                sumx = 0
                sumy = 0
                count = 0
                for name in grid.nodes():
                    if grid.nodes[name]['district']==i+1+(grid_size*grid_size):
                        sumx = sumx+name[0]
                        sumy = sumy+name[1]
                        count = count+1
                district_centers_new = district_centers_new+[[round(sumx/count), round(sumy/count)]]

            district_centers = district_centers_new
        # need to set up nodes once and reuse in iterations
        start_nodes, end_nodes, capacities, costs, supplies, source, sink, pops, total_pop=\
            setup(districts,grid_size,district_centers,pops)

        Block_Assignments = optimize(start_nodes, end_nodes, capacities, costs, supplies, source, sink, grid, grid_size)

        dist_area = {}
        district_dict = {}
        edge_boundary = {}



        for j in range(districts):
            pop_by_district = Block_Assignments.loc[Block_Assignments['DIST_NO']==j+1+(grid_size*grid_size),
                                                'ACTUAL_POP'].sum()
            dist_area.setdefault(j+1+(grid_size*grid_size),[]).append(pop_by_district/4)
 

        for node in grid.nodes():
            # create a dict with districts and assiciated nodes in them
            district_id = grid.nodes[node]['district']
            district_dict.setdefault(district_id,[]).append(node)
            # used for creating a df  for NN, row is position represents grid location, 
            # value in row represents party pref, meant to paint a picture of homogeneity for the NN.
            voter_pref = grid.nodes[node]["voter_pref"] 
            state_voterPrefs_as_row.append(voter_pref)
            

        party_count = 0
        for node in grid.nodes():
            district_id = grid.nodes[node]['district']
            win_count[district_id] = {}
            if party_count != 2:
                for node in grid.nodes():
                    voter_pref = grid.nodes[node]["voter_pref"]
                    win_count[district_id][voter_pref] = 0 
                    if (int(voter_pref) == 1.0) or (int(voter_pref) == 0.0):
                        party_count += 1
            if len(win_count) == dist_num:
                break;

        


        for node in grid.nodes():
            district_id = grid.nodes[node]['district']
            voter_pref = grid.nodes[node]["voter_pref"]             
            if str(voter_pref) == '1.0':
                win_count[district_id][voter_pref] += 1 
            if str(voter_pref) == '0.0':
                win_count[district_id][voter_pref] += 1 
        

    
        win_count = find_metrics(win_count)
        
        
        mino_win_count = 0

        dist_perimeter = {}  # Note the correct spelling here
        for key in district_dict:
            num_of_edge = 0
            District_subgraphs = district_dict[key]
            sub = grid.subgraph(District_subgraphs)
            for node in sub.nodes():
                edge_coming_out = sub.degree(node)
                if int(edge_coming_out) < 4:
                    num_of_edge += 1
            dist_perimeter[key] = num_of_edge  # Note the correct spelling here

        for key in dist_area:
            win_count[key]["area"] = dist_area[key][0]  # Note: accessing first element of the list
            win_count[key]["perimeter"] = dist_perimeter[key]  # Note the correct spelling here

        pp_compactness = get_pp_compactness(dist_perimeter, dist_area)  # Note the correct spelling here


        for key in pp_compactness:
            win_count[key]["pp_compactness"] = pp_compactness[key][0]
            
        district_data_each_iteration[l] = [win_count, l]
        pp_each_iter[l] = pp_compactness

        for item in win_count:
            mino_win_count = win_count[item]['win_count_0']
            break

        #print graphics
            
        if mino_win_count > 0 and print_graphics == 1:
            
            
            # Create a unique identifier

            unique_id = f"n{n}_k{districts}_step{l}_eg{win_count[list(win_count.keys())[0]]['eg']:.4f}"
            
            # Save district map
            newdir = f'graphs2/Districts_{unique_id}.png'
            plt.figure()
            nx.draw(grid, pos={x:x for x in grid.nodes()}, node_shape='s', node_size=20,
                    node_color=[grid.nodes[name]['district'] for name in grid.nodes()], cmap='Paired', with_labels=False)
            plt.savefig(newdir)
            plt.close()

            # Save voter preference map
            newdir1 = f'graphs2/VoterPreference_{unique_id}.png'
            plt.figure()
            nx.draw(grid, pos={x:x for x in grid.nodes()}, node_shape='s', node_size=20,
                    node_color=[grid.nodes[name]['voter_pref'] for name in grid.nodes()], cmap='Paired', with_labels=False)
            plt.savefig(newdir1)
            plt.close()
        
       

        for key in district_dict:
            node_list = district_dict[key]
            for node in node_list:
                edge_out_of_node = grid.edges(node)
                edge_num = len(edge_out_of_node)
                if edge_num < 4:
                    edge_boundary.setdefault(key,[]).append(node)


        dist_peremeter = {}

        #Print each dist
        for key in district_dict:
            num_of_edge = 0
            District_subgraphs = district_dict[key]
            sub = grid.subgraph(District_subgraphs)
            edge_boundary.setdefault(key,[]).append(num_of_edge)
            for node in sub.nodes():
                edge_coming_out = sub.degree(node)
                if int(edge_coming_out) < 4:
                    num_of_edge = num_of_edge+1
                    dist_peremeter.update({key : num_of_edge})



            

        dist_key = list(win_count.keys())
        state_voterPrefs_as_row.append(win_count[dist_key[0]]['pOverP_a'])
        state_voterPrefs_as_row.append(win_count[dist_key[0]]['pOverP_b'])
        
            
        pp_each_iter[l] = pp_compactness
        district_data_each_iteration.setdefault(l,[]).append(win_count)
            
        if log_flag == 1: 
            print('out dist assign')
    
    
    return district_data_each_iteration, pp_each_iter


In [77]:
def find_metrics(win_count):
    """
    Inputs: dictionary of win lose counts for parties within each district
    Purpose: find efficiency gap of district assignments
    Output: eg of districts packaged as dictionary
    """
    votes_1 = 0
    votes_0 = 0
    district_data = win_count
    wasted_votes = []
    total_win_a = 0
    total_win_b = 0
    total_votes_state = 0
    #imp_flag = 0 #set to 0 means immposiblity has not occured
    state_votes_a = 0
    state_votes_b = 0
    P_a = 0 #state_votes_a/total_votes_state
    P_b = 0
    p_a = 0 #total_win_a/state_count
    p_b = 0
    state_count = 0

    for key in win_count:
        state_count += 1
        votes_1 = win_count[key][1.0]
        votes_0 = win_count[key][0.0]
        state_votes_a += votes_0
        state_votes_b += votes_1
        totalVotes = votes_1 + votes_0
        total_votes_state += totalVotes
        
        if votes_1 > votes_0:
            half = math.ceil(totalVotes / 2)
            wasted_1 = votes_1 - half
            total_win_b += 1
        else:
            wasted_1 = votes_1

        if votes_1 < votes_0:
            half = math.ceil(totalVotes / 2)
            wasted_0 = votes_0 - half
            total_win_a += 1

        else:
            wasted_0 = votes_0
        
        wasted_total = wasted_0 - wasted_1
        
        wasted_votes.append(wasted_total)
        # if imp_flag == 1:
        #     district_data[key]['imp_flag'] = 1
        # else:
        #     district_data[key]['imp_flag'] = 0
    
    P_a =  state_votes_a/total_votes_state # vote percent
    P_b =  state_votes_b/total_votes_state
    p_a =  total_win_a/state_count # dist win percent
    p_b =  total_win_b/state_count

    if p_a == 0:
        pOverP_a = 0
    else:
        pOverP_a = P_a/p_a#p_a/P_a
    if p_b == 0:
        pOverP_b = 0
    else:
        pOverP_b = P_b/p_b
    sum_wv =sum(wasted_votes)
    eg = (1/total_votes_state)*sum_wv
    for key in win_count:
        district_data[key]['eg'] = eg
        district_data[key]['win_count_0'] = total_win_a
        district_data[key]['win_count_1'] = total_win_b
        district_data[key]['total_votes'] = total_votes_state
        district_data[key]['state_votes_a'] = state_votes_a
        district_data[key]['state_votes_b'] = state_votes_b
        district_data[key]['P_a'] = P_a
        district_data[key]['P_b'] = P_b
        district_data[key]['p_a'] = p_a
        district_data[key]['p_b'] = p_b
        district_data[key]['pOverP_a'] = pOverP_a
        district_data[key]['pOverP_b'] = pOverP_b
    
    return district_data 
    

In [78]:
def step_five_finder(delta, gamma, dists_num, p_0, n, c, r):
    # Implementation of step_five_finder function
    proportion_a = math.floor((c * r) * p_0)
    proportion_b = (c*r) - proportion_a
    left_side = proportion_b / proportion_a
    F = math.sqrt((1-delta)/(2*dists_num))
    numerator = (F**2)-(4*3.14*gamma**(-1)*F*math.sqrt(2)*(1/n))-(8*((3.14)**2)*(gamma**(-1))*(1/n)**2)
    denominator = (F**2)+(4*3.14*gamma**(-1)*F*math.sqrt(2)*(1/n))+(8*((3.14)**2)*(gamma**(-1))*(1/n)**2)
    right_side = numerator/denominator
    return left_side, right_side

def refined_step_five_finder(delta, gamma, dists_num, p_0, n, c, r):
    # Implementation of refined_step_five_finder function
    proportion_a = math.floor((c * r) * p_0)
    proportion_b = (c*r) - proportion_a
    left_side = proportion_b / proportion_a
    F = math.sqrt((1-delta)/(2*dists_num))
    numerator = 2*(1/n)*(math.pi)
    denominator = (4*gamma)*(math.sqrt((1-delta)/(dists_num+1)))*n
    right_side = numerator/denominator
    return left_side, right_side

In [79]:
def find_winners(win_count):
    partyOneWin = 0
    partyZeroWin = 0
    for iteration in win_count:
        for districts in win_count[iteration]:
            for district in districts:
                for party in districts[district]:

                    if party == 1.0 :
                        if str(districts[district][party]) == '{}':
                            partyOneCount = 0
                        else:
                            partyOneCount = int(str(districts[district][party]))

                    if party == 0.0:
                        partyZeroCount = int(str(districts[district][party]))


                if partyOneCount > partyZeroWin:
                    partyOneCount += 1
                if partyOneCount < partyZeroWin:
                    partyZeroCount += 1
    return partyOneWin, partyZeroWin

In [80]:
def create_dataFrame(win_count, extra, pp_each_iter):
    data = []
    for iteration, iteration_data in win_count.items():
        district_data = iteration_data[0]  # The first element is the district data
        iteration_number = iteration_data[1]  # The second element is the iteration number
        
        # Get the pp_compactness for this iteration
        pp_compactness_iter = pp_each_iter.get(iteration, {})
        
        for district, district_info in district_data.items():
            votes0 = district_info.get(0.0, 0)
            votes1 = district_info.get(1.0, 0)
            area = district_info.get('area', [0])[0] if isinstance(district_info.get('area'), list) else district_info.get('area', 0)
            perimeter = district_info.get('perimeter', 0)
            pp_compactness = pp_compactness_iter.get(district, [0])[0] if isinstance(pp_compactness_iter.get(district), list) else pp_compactness_iter.get(district, 0)
            #imp_flag = district_info.get('imp_flag', 0)
            eg = district_info.get('eg', 0)
            win_count_0 = district_info.get('win_count_0', 0)
            win_count_1 = district_info.get('win_count_1', 0)
            total_votes = district_info.get('total_votes', 0)
            state_votes_0 = district_info.get('state_votes_a', 0)
            state_votes_1 = district_info.get('state_votes_b', 0)
            P_0 = district_info.get('P_a', 0)
            P_1 = district_info.get('P_b', 0)
            p_0 = district_info.get('p_a', 0)
            p_1 = district_info.get('p_b', 0)
            pOverP_0 = district_info.get('pOverP_a', 0)
            pOverP_1 = district_info.get('pOverP_b', 0)

            row = [votes0, votes1, area, perimeter, pp_compactness, eg, win_count_0, win_count_1, 
                   total_votes, state_votes_0, state_votes_1, P_0, P_1, p_0, p_1, pOverP_0, pOverP_1, 
                   iteration_number, district]
            row.extend(extra)
            data.append(row)

    columns = ['votes_0', 'votes_1', 'area', 'perimeter', 'pp_compactness', 'eg', 'win_count_0', 'win_count_1', 
               'total_votes', 'state_votes_0', 'state_votes_1', 'P_0', 'P_1', 'p_0', 'p_1', 'pOverP_0', 'pOverP_1', 
               'iteration', 'district', 'n', 'L', 'K', 'proportion_0', 'left_side', 'right_side', 
               'refined_left_side', 'refined_right_side', 'delta', 'gamma', 'c*r']  


    df = pa.DataFrame(data, columns=columns)
    return df

In [81]:
# # Define constants
# p_0 = 0.5  # proportion of party B voters
# c = 3  # column count within square
# r = 3  # row count within square
# delta = 0.05
# gamma = 0.2
# iterations = 10

# # Generate all combinations of n and k from 1 to 15
# examples = [(n, k) for n in range(1, 16) for k in range(1, 16)]

# main = pa.DataFrame(columns=['votesA', 'votesB', 'area', 'perimeter', 'pp_compactness', 'imp_flag', 'eg', 
#                              'win_count_a', 'win_count_b', 'total_votes', 'state_votes_a', 'state_votes_b',
#                              'P_a', 'P_b', 'p_a', 'p_b', 'pOverP_a', 'pOverP_b', 'iteration', 'district', 
#                              'n', 'L', 'K', 'proportion_a', 'left_side', 'right_side', 
#                              'refined_left_side', 'refined_right_side'])

# for n, k in examples:
#     grid_length = n * c  # entire grid length
    
#     left_side, right_side = step_five_finder(delta, gamma, k, p_0, n, c, r)
#     refined_left_side, refined_right_side = refined_step_five_finder(delta, gamma, k, p_0, n, c, r)
    
#     try:
#         district_data, pp_each_iter = dist_assgin(iterations, grid_length, k, n, r, c, p_0)
        
#         extra = [n, (c*r), k, p_0, left_side, right_side, refined_left_side, refined_right_side]
#         df = create_dataFrame(district_data, extra, pp_each_iter)
#         main = pa.concat([df, main], ignore_index=True)
        
#         print(f"Completed simulation for n={n}, k={k}")
#     except Exception as e:
#         print(f"Error in simulation for n={n}, k={k}: {str(e)}")
    
#     # Optional: Print progress every 10 simulations
#     if (n * 15 + k) % 10 == 0:
#         print(f"Progress: Completed {n * 15 + k} out of {15 * 15} simulations")

# # Save the results
# csvDir = 'simulation_results_n1_15_k1_15.csv'
# main.to_csv(csvDir, index=False)
# print("Simulation completed. Results saved to", csvDir)

In [83]:
import time
from tqdm import tqdm

def run_simulation(params):
    n, k, delta, gamma = params
    grid_length = n * c  # entire grid length
    print(f"Starting simulation for n={n}, k={k}, delta={delta}, gamma={gamma}")

    L = math.sqrt(c * r)
    left_side, right_side = step_five_finder(delta, gamma, k, p_0, n, c, r)
    refined_left_side, refined_right_side = refined_step_five_finder(delta, gamma, k, p_0, n, c, r)
    
    district_data, pp_each_iter = dist_assgin(iterations, grid_length, k, n, r, c, p_0)
    
    extra = [n, L, k, p_0, left_side, right_side, refined_left_side, refined_right_side, delta, gamma, c*r]
    df = create_dataFrame(district_data, extra, pp_each_iter)
    
    return df

if __name__ == '__main__':
    # Define constants
    p_0 = 0.5
    c = 3
    r = 3
    iterations = 100

    # Generate all combinations of n, k, delta, and gamma
    examples = [
        (n, k, delta, gamma)
        for n in range(1, 16)
        for k in range(1, 16)
        for delta in [round(d, 2) for d in np.arange(0.01, 0.10, 0.01)]
        for gamma in [round(g, 1) for g in np.arange(0.1, 0.8, 0.1)]
    ]
    
    start_time = time.time()
    
    results = []
    for params in tqdm(examples):
        try:
            n, k, delta, gamma = params
            result = run_simulation((n, k, delta, gamma))
            if result is not None and not result.empty:
                results.append(result)
        except Exception as e:
            print(f"Error in simulation for n={n}, k={k}, delta={delta}, gamma={gamma}: {str(e)}")
            continue

    if results:
        main = pa.concat(results, ignore_index=True)
        
        end_time = time.time()
        print(f"Total execution time: {end_time - start_time:.2f} seconds")
        
        csvDir = 'simulation_data_varied_params.csv'
        main.to_csv(csvDir, index=False)
        print(f"Results saved to {csvDir}")
    else:
        print("No valid results were produced.")

    print("Simulation completed.")

  0%|          | 1/225 [00:00<01:24,  2.65it/s]

Starting simulation for n=1, k=1
Starting simulation for n=1, k=2


  1%|          | 2/225 [00:00<01:26,  2.59it/s]

Starting simulation for n=1, k=3


  1%|▏         | 3/225 [00:01<01:27,  2.54it/s]

...

 90%|█████████ | 203/225 [3:47:30<1:16:02, 207.39s/it]

Starting simulation for n=14, k=9


 90%|█████████ | 203/225 [3:48:31<24:46, 67.55s/it]   


KeyboardInterrupt: 

: 